In [1]:
import serial
import numpy as np
from time import sleep
import datetime
import time
import csv
# Serial connection settings
port = 'COM5'  # Replace with the appropriate serial port
baud_rate = 115200

In [2]:
from Kalman import KalmanAngle
import time
import math

In [3]:
kalmanX1 = KalmanAngle()
kalmanY1 = KalmanAngle()
kalmanX2 = KalmanAngle()
kalmanY2 = KalmanAngle()
kalmanX3 = KalmanAngle()
kalmanY3 = KalmanAngle()
RestrictPitch = True	#Comment out to restrict roll to ±90deg instead - please read: http://www.freescale.com/files/sensors/doc/app_note/AN3461.pdf
radToDeg = 57.2957786
kalAngleX1 = 0
kalAngleY1 = 0
kalAngleX2 = 0
kalAngleY2 = 0
kalAngleX3 = 0
kalAngleY3 = 0
L1 = 0
L2 = 3
L3 = 3.5
L4 = 2

In [ ]:
ser = serial.Serial(port, baud_rate)
sleep(5)
ser.write(b"rrrrrrrrrr\r\n")
ser.flush()

csv_file = open('echo1.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'KalmanXSensor1', 'KalmanYSensor1', 'KalmanXSensor2', 'KalmanYSensor2', 'KalmanXSensor3', 'KalmanYSensor3', 'PIP-X', 'PIP-Y', 'PIP-Z', 'DIP-X', 'DIP-Y', 'DIP-Z', 'Tip-X', 'Tip-Y', 'Tip-Z'])

try:    

    timer = time.time()

    while True:
        dt = time.time() - timer
        timer = time.time()
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        
        # Read data from serial port
        if ser.in_waiting > 0:
            values = [None]*3
            while None in values:
                line = ser.read_until()
                if not line.startswith(b"Sensor"):
                    print("omitted", line)
                    continue
                #print(line, [7])
                values[int(line[7:8])] = line[8:].strip().split(b',')
            #data = ser.readline().decode().strip().split(',')
            (ax1, ay1, az1, gx1, gy1, gz1), (ax2, ay2, az2, gx2, gy2, gz2), (ax3, ay3, az3, gx3, gy3, gz3) = [map(float, Sensor) for Sensor in values]
            if (RestrictPitch):
                roll1 = math.atan2(ay1,az1) * radToDeg
                pitch1 = math.atan(-ax1/math.sqrt((ay1**2)+(az1**2))) * radToDeg
                roll2 = math.atan2(ay2,az2) * radToDeg
                pitch2 = math.atan(-ax2/math.sqrt((ay2**2)+(az2**2))) * radToDeg
                roll3 = math.atan2(ay3,az3) * radToDeg
                pitch3 = math.atan(-ax3/math.sqrt((ay3**2)+(az3**2))) * radToDeg
                    
            else:
                roll1 = math.atan(ay1/math.sqrt((ax1**2)+(az1**2))) * radToDeg
                pitch1 = math.atan2(-ax1,az1) * radToDeg
                roll2 = math.atan(ay2/math.sqrt((ax2**2)+(az2**2))) * radToDeg
                pitch2 = math.atan2(-ax2,az2) * radToDeg
                roll3 = math.atan(ay3/math.sqrt((ax3**2)+(az3**2))) * radToDeg
                pitch3 = math.atan2(-ax3,az3) * radToDeg
            gyroX1Rate = gx1/131
            gyroY1Rate = gy1/131
            compAngleX1 = roll1;
            compAngleY1 = pitch1;
            gyroX2Rate = gx2/131
            gyroY2Rate = gy2/131
            compAngleX2 = roll2;
            compAngleY2 = pitch2;
            gyroX3Rate = gx3/131
            gyroY3Rate = gy3/131
            compAngleX3 = roll3;
            compAngleY3 = pitch3;
                
            if (RestrictPitch):

                if((roll1 < -90 and kalAngleX1 >90) or (roll1 > 90 and kalAngleX1 < -90)):
                    kalmanX1.setAngle(roll1)
                    complAngleX1 = roll1
                    kalAngleX1 = roll1
                    gyroXAngle1 = roll1
                        
                else:
                    kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)

                if(abs(kalAngleX1)>90):
                    gyroY1Rate  = -gyroY1Rate
                    kalAngleY1  = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)

            else:
                if((pitch1 < -90 and kalAngleY1 >90) or (pitch1 > 90 and kalAngleY1 < -90)):
                    kalmanY1.setAngle(pitch1)
                    complAngleY1 = pitch1
                    kalAngleY1   = pitch1
                    gyroYAngle1  = pitch1
                else:
                    kalAngleY1 = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)

                if(abs(kalAngleY1)>90):
                    gyroX1Rate  = -gyroX1Rate
                    kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)
        
            gyroXAngle1 = gyroX1Rate * dt
            gyroYAngle1 = gyroY1Rate * dt

            compAngleX1 = 0.93 * (compAngleX1 + gyroX1Rate * dt) + 0.07 * roll1
            compAngleY1 = 0.93 * (compAngleY1 + gyroY1Rate * dt) + 0.07 * pitch1
                
            if ((gyroXAngle1 < -180) or (gyroXAngle1 > 180)):
                gyroXAngle1 = kalAngleX1
            if ((gyroYAngle1 < -180) or (gyroYAngle1 > 180)):
                gyroYAngle1 = kalAngleY1




            if (RestrictPitch):

                if((roll2 < -90 and kalAngleX2 >90) or (roll2 > 90 and kalAngleX2 < -90)):
                    kalmanX2.setAngle(roll2)
                    complAngleX2 = roll2
                    kalAngleX2 = roll2
                    gyroXAngle2 = roll2
                        
                else:
                    kalAngleX2 = kalmanX2.getAngle(roll2,gyroX2Rate,dt)

                if(abs(kalAngleX2)>90):
                    gyroY2Rate  = -gyroY2Rate
                    kalAngleY2  = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

            else:
                if((pitch2 < -90 and kalAngleY2 >90) or (pitch2 > 90 and kalAngleY2 < -90)):
                    kalmanY2.setAngle(pitch2)
                    complAngleY2 = pitch2
                    kalAngleY2   = pitch2
                    gyroYAngle2  = pitch2
                else:
                    kalAngleY2 = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

                if(abs(kalAngleY2)>90):
                    gyroX2Rate  = -gyroX2Rate
                    kalAngleX2 = kalmanX2.getAngle(roll1,gyroX1Rate,dt)
            gyroXAngle2 = gyroX2Rate * dt
            gyroYAngle2 = gyroY2Rate * dt

            compAngleX2 = 0.93 * (compAngleX2 + gyroX2Rate * dt) + 0.07 * roll2
            compAngleY2 = 0.93 * (compAngleY2 + gyroY2Rate * dt) + 0.07 * pitch2
                
            if ((gyroXAngle2 < -180) or (gyroXAngle2 > 180)):
                gyroXAngle2 = kalAngleX2
            if ((gyroYAngle2 < -180) or (gyroYAngle2 > 180)):
                gyroYAngle2 = kalAngleY2


            if (RestrictPitch):

                if((roll3 < -90 and kalAngleX3 >90) or (roll3 > 90 and kalAngleX3 < -90)):
                    kalmanX3.setAngle(roll3)
                    complAngleX3 = roll3
                    kalAngleX3 = roll3
                    gyroXAngle3 = roll3
                        
                else:
                    kalAngleX3 = kalmanX3.getAngle(roll3,gyroX3Rate,dt)

                if(abs(kalAngleX3)>90):
                    gyroY3Rate  = -gyroY3Rate
                    kalAngleY3  = kalmanY3.getAngle(pitch3,gyroY3Rate,dt)

            else:
                if((pitch3 < -90 and kalAngleY3 >90) or (pitch3 > 90 and kalAngleY3 < -90)):
                    kalmanY3.setAngle(pitch3)
                    complAngleY3 = pitch3
                    kalAngleY3   = pitch3
                    gyroYAngle3  = pitch3
                else:
                    kalAngleY3 = kalmanY3.getAngle(pitch3,gyroY3Rate,dt)

                if(abs(kalAngleY3)>90):
                    gyroX3Rate  = -gyroX3Rate
                    kalAngleX3 = kalmanX3.getAngle(roll3,gyroX3Rate,dt)
            gyroXAngle3 = gyroX3Rate * dt
            gyroYAngle3 = gyroY3Rate * dt


            print("KAngle X1: " + str(kalAngleX1)+"   " +"KAngle Y1: " + str(kalAngleY1)+"   " +"KAngle X2: " + str(kalAngleX2)+"   " +"KAngle Y2: " + str(kalAngleY2)+"  "+"KAngle X3: " + str(kalAngleX3)+"   " +"KAngle Y3: " + str(kalAngleY3))
            
            kalAngleX1_rad = (kalAngleX1 * 3.142)/180
            kalAngleX2_rad = (kalAngleX2 * 3.142)/180
            kalAngleX3_rad = (kalAngleX3 * 3.142)/180

            J1x = 0
            J1y = 0
            J1z = 0
             #FINGER 1
            F1J2x = L2 * np.cos(kalAngleX1_rad) * np.cos(kalAngleX2_rad)
            F1J2y = L2 * np.sin(kalAngleX1_rad) * np.cos(kalAngleX2_rad)
            F1J2z = L2 * np.sin(kalAngleX2_rad)

            F1J3x = np.cos(kalAngleX1_rad) * (L2 * np.cos(kalAngleX2_rad) + L3 * np.cos(kalAngleX2_rad + kalAngleX3_rad))
            F1J3y = np.sin(kalAngleX1_rad) * (L2 * np.cos(kalAngleX2_rad) + L3 * np.cos(kalAngleX2_rad + kalAngleX3_rad))
            F1J3z = L2 * np.sin(kalAngleX2_rad) + L3 * np.sin(kalAngleX2_rad + kalAngleX3_rad)

            F1J4x = (np.cos(kalAngleX1_rad)) * (L2 * np.cos(kalAngleX2_rad) + L3 * np.cos(kalAngleX2_rad + kalAngleX3_rad) + L4 * np.cos(kalAngleX2_rad + kalAngleX3_rad))
            F1J4y = (np.sin(kalAngleX1_rad)) * (L2 * np.sin(kalAngleX2_rad) + L3 * np.sin(kalAngleX2_rad + kalAngleX3_rad) + L4 * np.sin(kalAngleX2_rad + kalAngleX3_rad))
            F1J4z = L2 * np.sin(kalAngleX3_rad) + L3 * np.sin(kalAngleX2_rad + kalAngleX3_rad) + L4 * np.sin(kalAngleX3_rad)
            
            
            data_timestamp = [current_time, kalAngleX1, kalAngleY1, kalAngleX2, kalAngleY2, kalAngleX3, kalAngleY3, F1J2x, F1J2y, F1J2z, F1J3x, F1J3y, F1J3z, F1J4x, F1J4y, F1J4z]
            csv_writer.writerow(data_timestamp)

            

                
except KeyboardInterrupt:
    ser.close()

In [43]:
ser = serial.Serial(port, baud_rate)
sleep(5)
ser.write(b"rrrrrrrrrr\r\n")
ser.flush()

# Create a CSV file and write headers
csv_file = open('EXP3.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Kalman_Roll1', 'Kalman_Pitch1', 'Kalman_Roll2', 'Kalman_Pitch2', ])

try:
    while not (line := ser.read_until()).startswith(b'Sensor'):
        pass
    *_, ax1, _, ay1, _, az1, _, gx1, _, gy1, _, gz1 = line.rsplit(maxsplit=12)
    *_, ax2, _, ay2, _, az2, _, gx2, _, gy2, _, gz2 = ser.read_until().rsplit(maxsplit=12)
    timer = time.time()
    for _ in range(25):
        dt = time.time() - timer
        timer = time.time()

        ax1, ay1, az1, gx1, gy1, gz1 = map(float, (ax1, ay1, az1, gx1, gy1, gz1))
        ax2, ay2, az2, gx2, gy2, gz2 = map(float, (ax2, ay2, az2, gx2, gy2, gz2))

        if (RestrictPitch):
            roll1 = math.atan2(ay1,az1) * radToDeg
            pitch1 = math.atan(-ax1/math.sqrt((ay1**2)+(az1**2))) * radToDeg
            roll2 = math.atan2(ay2,az2) * radToDeg
            pitch2 = math.atan(-ax2/math.sqrt((ay2**2)+(az2**2))) * radToDeg
        else:
            roll1 = math.atan(ay1/math.sqrt((ax1**2)+(az1**2))) * radToDeg
            pitch1 = math.atan2(-ax1,az1) * radToDeg
            roll2 = math.atan(ay2/math.sqrt((ax2**2)+(az2**2))) * radToDeg
            pitch2 = math.atan2(-ax2,az2) * radToDeg
        gyroX1Rate = gx1/131
        gyroY1Rate = gy1/131
        gyroX2Rate = gx2/131
        gyroY2Rate = gy2/131


        if (RestrictPitch):

            if((roll1 < -90 and kalAngleX1 >90) or (roll1 > 90 and kalAngleX1 < -90)):
                kalmanX1.setAngle(roll1)
                complAngleX1 = roll1
                kalAngleX1 = roll1
                gyroXAngle1 = roll1
                        
            else:
                kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)

            if(abs(kalAngleX1)>90):
                gyroY1Rate  = -gyroY1Rate
                kalAngleY1  = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)
        else:
            if((pitch1 < -90 and kalAngleY1 >90) or (pitch1 > 90 and kalAngleY1 < -90)):
                kalmanY1.setAngle(pitch1)
                complAngleY1 = pitch1
                kalAngleY1   = pitch1
                gyroYAngle1  = pitch1
            else:
                kalAngleY1 = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)

            if(abs(kalAngleY1)>90):
                gyroX1Rate  = -gyroX1Rate
                kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)
        
        gyroXAngle1 = gyroX1Rate * dt
        gyroYAngle1 = gyroY1Rate * dt

            
        if ((gyroXAngle1 < -180) or (gyroXAngle1 > 180)):
            gyroXAngle1 = kalAngleX1
        if ((gyroYAngle1 < -180) or (gyroYAngle1 > 180)):
            gyroYAngle1 = kalAngleY1

        
        if (RestrictPitch):

            if((roll2 < -90 and kalAngleX2 >90) or (roll2 > 90 and kalAngleX2 < -90)):
                kalmanX2.setAngle(roll2)
                complAngleX2 = roll2
                kalAngleX2 = roll2
                gyroXAngle2 = roll2
                        
            else:
                kalAngleX2 = kalmanX2.getAngle(roll2,gyroX2Rate,dt)

            if(abs(kalAngleX2)>90):
                gyroY2Rate  = -gyroY2Rate
                kalAngleY2  = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

        else:
            if((pitch2 < -90 and kalAngleY2 >90) or (pitch2 > 90 and kalAngleY2 < -90)):
                kalmanY2.setAngle(pitch2)
                complAngleY2 = pitch2
                kalAngleY2   = pitch2
                gyroYAngle2  = pitch2
            else:
                 kalAngleY2 = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

            if(abs(kalAngleY2)>90):
                gyroX2Rate  = -gyroX2Rate
                kalAngleX2 = kalmanX2.getAngle(roll1,gyroX1Rate,dt)
        gyroXAngle2 = gyroX2Rate * dt
        gyroYAngle2 = gyroY2Rate * dt
        
        if ((gyroXAngle2 < -180) or (gyroXAngle2 > 180)):
            gyroXAngle2 = kalAngleX2
        if ((gyroYAngle2 < -180) or (gyroYAngle2 > 180)):
            gyroYAngle2 = kalAngleY2

        robot_data = [kalAngleX1, kalAngleY1, kalAngleX2, kalAngleY2]
        csv_writer.writerow(robot_data)

        *_, ax1, _, ay1, _, az1, _, gx1, _, gy1, _, gz1 = ser.read_until().rsplit(maxsplit=12)
        *_, ax2, _, ay2, _, az2, _, gx2, _, gy2, _, gz2 = ser.read_until().rsplit(maxsplit=12)
except KeyboardInterrupt:
    ser.close()
csv_file.close()

In [7]:
def get_next_sensor_data():
    while not (line := ser.read_until()).startswith(b'Sensor'):
        pass

    *_, ax, _, ay, _, az, _, gx, _, gy, _, gz = line.rsplit(maxsplit=12)

    return tuple(map(float, (ax, ay, az, gx, gy, gz)))

In [ ]:
ser = serial.Serial(port, baud_rate)
sleep(5)
ser.write(b"rrrrrrrrrr\r\n")
ser.flush()

# Create a CSV file and write headers
csv_file = open('Datasets/me1.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Increment', 'az1', 'Az1_Angle', 'KalAngleX1', 'az2', 'Az2_Angle', 'KalAngleX2'])


try:
    while not (line := ser.read_until()).startswith(b'Sensor'):
        pass
    *_, ax1, _, ay1, _, az1, _, gx1, _, gy1, _, gz1 = line.rsplit(maxsplit=12)
    *_, ax2, _, ay2, _, az2, _, gx2, _, gy2, _, gz2 = ser.read_until().rsplit(maxsplit=12)
    
    for _ in range(25):
        dt = 0.08
        increment = _ * 7.2
        

        ax1, ay1, az1, gx1, gy1, gz1 = get_next_sensor_data()
        az1_angle = (((az1/16384) * 9.8)/0.06) * dt * dt  # Calculate angle from acceleration
        ax1_angle = (((ax1/16384) * 9.8)/0.06) * dt * dt

        ax2, ay2, az2, gx2, gy2, gz2 = get_next_sensor_data()
        az2_angle = (((az2/16384) * 9.8)/0.06) * dt * dt  # Calculate angle from acceleration
        ax2_angle = (((ax2/16384) * 9.8)/0.06) * dt * dt

        if (RestrictPitch):
            roll1 = math.atan2(ay1,az1) * radToDeg
            pitch1 = math.atan2(-ax1,az1) * radToDeg
            pitch1 = math.atan(-ax1/math.sqrt((ay1**2)+(az1**2))) * radToDeg
            roll2 = math.atan2(ay2,az2) * radToDeg
            pitch2 = math.atan(-ax2/math.sqrt((ay2**2)+(az2**2))) * radToDeg
        else:
            roll1 = math.atan(ay1/math.sqrt((ax1**2)+(az1**2))) * radToDeg
            pitch1 = math.atan2(-ax1,az1) * radToDeg
            roll2 = math.atan(ay2/math.sqrt((ax2**2)+(az2**2))) * radToDeg
            pitch2 = math.atan2(-ax2,az2) * radToDeg
        gyroX1Rate = gx1/131
        gyroY1Rate = gy1/131
        gyroX2Rate = gx2/131
        gyroY2Rate = gy2/131


        if (RestrictPitch):

            if((roll1 < -90 and kalAngleX1 >90) or (roll1 > 90 and kalAngleX1 < -90)):
                kalmanX1.setAngle(roll1)
                complAngleX1 = roll1
                kalAngleX1 = roll1
                gyroXAngle1 = roll1
                        
            else:
                kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)

            if(abs(kalAngleX1)>90):
                gyroY1Rate  = -gyroY1Rate
                kalAngleY1  = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)
        else:
            if((pitch1 < -90 and kalAngleY1 >90) or (pitch1 > 90 and kalAngleY1 < -90)):
                kalmanY1.setAngle(pitch1)
                complAngleY1 = pitch1
                kalAngleY1   = pitch1
                gyroYAngle1  = pitch1
            else:
                kalAngleY1 = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)

            if(abs(kalAngleY1)>90):
                gyroX1Rate  = -gyroX1Rate
                kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)
        
        gyroXAngle1 = gyroX1Rate * dt
        gyroYAngle1 = gyroY1Rate * dt

            
        if ((gyroXAngle1 < -180) or (gyroXAngle1 > 180)):
            gyroXAngle1 = kalAngleX1
        if ((gyroYAngle1 < -180) or (gyroYAngle1 > 180)):
            gyroYAngle1 = kalAngleY1

        
        if (RestrictPitch):

            if((roll2 < -90 and kalAngleX2 >90) or (roll2 > 90 and kalAngleX2 < -90)):
                kalmanX2.setAngle(roll2)
                complAngleX2 = roll2
                kalAngleX2 = roll2
                gyroXAngle2 = roll2
                        
            else:
                kalAngleX2 = kalmanX2.getAngle(roll2,gyroX2Rate,dt)

            if(abs(kalAngleX2)>90):
                gyroY2Rate  = -gyroY2Rate
                kalAngleY2  = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

        else:
            if((pitch2 < -90 and kalAngleY2 >90) or (pitch2 > 90 and kalAngleY2 < -90)):
                kalmanY2.setAngle(pitch2)
                complAngleY2 = pitch2
                kalAngleY2   = pitch2
                gyroYAngle2  = pitch2
            else:
                 kalAngleY2 = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

            if(abs(kalAngleY2)>90):
                gyroX2Rate  = -gyroX2Rate
                kalAngleX2 = kalmanX2.getAngle(roll1,gyroX1Rate,dt)
        gyroXAngle2 = gyroX2Rate * dt
        gyroYAngle2 = gyroY2Rate * dt
        
        if ((gyroXAngle2 < -180) or (gyroXAngle2 > 180)):
            gyroXAngle2 = kalAngleX2
        if ((gyroYAngle2 < -180) or (gyroYAngle2 > 180)):
            gyroYAngle2 = kalAngleY2


        csv_writer.writerow([increment, az1, az1_angle, kalAngleX1, az2, az2_angle, kalAngleX2])

except KeyboardInterrupt:
    ser.close()
# except Exception as err:
#     print("Error: ", err)
csv_file.close()

In [57]:
import csv

def merge_csv(file1, file2, output_file):
    # Read data from the first CSV file
    with open(file1, 'r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        data1 = list(reader)

    # Read data from the second CSV file
    with open(file2, 'r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        data2 = list(reader)

    # Merge data
    merged_data = data1 + data2

    # Write merged data into a new CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(merged_data)

if __name__ == "__main__":
    file1 = 'Datasets/EXPONOROTATE.csv'  # Replace with the path to your first CSV file
    file2 = 'Datasets/EXPOROTATE.csv'  # Replace with the path to your second CSV file
    output_file = 'Datasets/MERGED.csv'  # Replace with the desired name for the merged CSV file

    merge_csv(file1, file2, output_file)
    print("CSV files merged successfully.")


CSV files merged successfully.


In [8]:
import re

SENSOR_DATA_re = re.compile(rb",\s+".join((rb"(-?\d+)",) * 12))


def get_next_sensors_data():
    while not (match := SENSOR_DATA_re.match(ser.read_until())):
        pass

    return tuple(map(int, match.groups()))


In [26]:
ser = serial.Serial(port, baud_rate)
sleep(5)
 
input("Enter any character and press enter")
ser.write(b"rrrrrrrrrr\r\n")
ser.flush()

# Create a CSV file and write headers
csv_file = open('Datasets/B100HZ.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(
    ['Timestamp', 'Increment', 'ax1', 'ay1', 'az1', 'gx1', 'gy1', 'gz1', 'ax2', 'ay2', 'az2', 'gx2', 'gy2', 'gz2']
)

try:
    for n in range(50):
        dt = 0.08
        increment = n * 7.2
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        csv_writer.writerow([current_time, increment, *get_next_sensors_data()])
        csv_file.flush()
except KeyboardInterrupt:
    ser.close()
finally:
    csv_file.close()

In [31]:
ser = serial.Serial(port, baud_rate)
sleep(5)


input("Enter any character and press enter")
ser.write(b"rrrrrrrrrr\r\n")
ser.flush()

# Create a CSV file and write headers
csv_file = open('Datasets/S0.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'Increment', 'az1', 'Az1_Angle', 'KalAngleX1', 'az2', 'Az2_Angle', 'KalAngleX2'])

try:
    for _ in range(25):
        dt = 0.08
        increment = _ * 7.2
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

        if not any (sensor_data := get_next_sensors_data()):
            csv_writer.writerow([current_time, increment, 0, 0, 0, 0, 0, 0])
            continue
        
        ax1, ay1, az1, gx1, gy1, gz1, ax2, ay2, az2, gx2, gy2, gz2 = sensor_data

        # Calculate angle from acceleration
        az1_angle = (((az1/16384) * 9.8)/0.06) * dt * dt
        ax1_angle = (((ax1/16384) * 9.8)/0.06) * dt * dt
        az2_angle = (((az2/16384) * 9.8)/0.06) * dt * dt
        ax2_angle = (((ax2/16384) * 9.8)/0.06) * dt * dt

        if (RestrictPitch):
            roll1 = math.atan2(ay1,az1) * radToDeg
            pitch1 = math.atan2(-ax1,az1) * radToDeg
            #pitch1 = math.atan(-ax1/math.sqrt((ay1**2)+(az1**2))) * radToDeg
            roll2 = math.atan2(ay2,az2) * radToDeg
            pitch2 = math.atan(-ax2/math.sqrt((ay2**2)+(az2**2))) * radToDeg
        else:
            roll1 = math.atan(ay1/math.sqrt((ax1**2)+(az1**2))) * radToDeg
            pitch1 = math.atan2(-ax1,az1) * radToDeg
            roll2 = math.atan(ay2/math.sqrt((ax2**2)+(az2**2))) * radToDeg
            pitch2 = math.atan2(-ax2,az2) * radToDeg
        gyroX1Rate = gx1/131
        gyroY1Rate = gy1/131
        gyroX2Rate = gx2/131
        gyroY2Rate = gy2/131

        if (RestrictPitch):

            if((roll1 < -90 and kalAngleX1 >90) or (roll1 > 90 and kalAngleX1 < -90)):
                kalmanX1.setAngle(roll1)
                complAngleX1 = roll1
                kalAngleX1 = roll1
                gyroXAngle1 = roll1
                        
            else:
                kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)

            if(abs(kalAngleX1)>90):
                gyroY1Rate  = -gyroY1Rate
                kalAngleY1  = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)
        else:
            if((pitch1 < -90 and kalAngleY1 >90) or (pitch1 > 90 and kalAngleY1 < -90)):
                kalmanY1.setAngle(pitch1)
                complAngleY1 = pitch1
                kalAngleY1   = pitch1
                gyroYAngle1  = pitch1
            else:
                kalAngleY1 = kalmanY1.getAngle(pitch1,gyroY1Rate,dt)

            if(abs(kalAngleY1)>90):
                gyroX1Rate  = -gyroX1Rate
                kalAngleX1 = kalmanX1.getAngle(roll1,gyroX1Rate,dt)
        
        gyroXAngle1 = gyroX1Rate * dt
        gyroYAngle1 = gyroY1Rate * dt

            
        if ((gyroXAngle1 < -180) or (gyroXAngle1 > 180)):
            gyroXAngle1 = kalAngleX1
        if ((gyroYAngle1 < -180) or (gyroYAngle1 > 180)):
            gyroYAngle1 = kalAngleY1

        if (RestrictPitch):

            if((roll2 < -90 and kalAngleX2 >90) or (roll2 > 90 and kalAngleX2 < -90)):
                kalmanX2.setAngle(roll2)
                complAngleX2 = roll2
                kalAngleX2 = roll2
                gyroXAngle2 = roll2
                        
            else:
                kalAngleX2 = kalmanX2.getAngle(roll2,gyroX2Rate,dt)

            if(abs(kalAngleX2)>90):
                gyroY2Rate  = -gyroY2Rate
                kalAngleY2  = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

        else:
            if((pitch2 < -90 and kalAngleY2 >90) or (pitch2 > 90 and kalAngleY2 < -90)):
                kalmanY2.setAngle(pitch2)
                complAngleY2 = pitch2
                kalAngleY2   = pitch2
                gyroYAngle2  = pitch2
            else:
                 kalAngleY2 = kalmanY2.getAngle(pitch2,gyroY2Rate,dt)

            if(abs(kalAngleY2)>90):
                gyroX2Rate  = -gyroX2Rate
                kalAngleX2 = kalmanX2.getAngle(roll1,gyroX1Rate,dt)
        gyroXAngle2 = gyroX2Rate * dt
        gyroYAngle2 = gyroY2Rate * dt
        
        if ((gyroXAngle2 < -180) or (gyroXAngle2 > 180)):
            gyroXAngle2 = kalAngleX2
        if ((gyroYAngle2 < -180) or (gyroYAngle2 > 180)):
            gyroYAngle2 = kalAngleY2

        csv_writer.writerow([current_time, increment, az1, az1_angle, kalAngleX1, az2, az2_angle, kalAngleX2])

except KeyboardInterrupt:
    ser.close()
#except ZeroDivisionError:
    #print("Error: Division by zero is not allowed.")
finally:
    csv_file.close()

In [32]:
# Kalman filter parameters for each sensor
class KalmanFilter:
    def __init__(self):
        self.Q_angle = 0.001  # Process noise variance for the accelerometer
        self.Q_bias = 0.003   # Process noise variance for the gyroscope bias
        self.R_measure = 0.03 # Measurement noise variance

        self.angle = 0.0      # The angle calculated by the Kalman filter
        self.bias = 0.0       # The gyroscope bias calculated by the filter
        self.rate = 0.0       # Unbiased rate

        self.P = np.zeros((2, 2))  # Error covariance matrix
        self.K = np.zeros(2)       # Kalman gain
        self.y = 0.0               # Angle difference
        self.S = 0.0               # Estimate error

    def update(self, new_angle, new_rate, dt):
        # Predict
        self.rate = new_rate - self.bias
        self.angle += dt * self.rate

        # Update covariance matrix
        self.P[0][0] += dt * (dt*self.P[1][1] - self.P[0][1] - self.P[1][0] + self.Q_angle)
        self.P[0][1] -= dt * self.P[1][1]
        self.P[1][0] -= dt * self.P[1][1]
        self.P[1][1] += self.Q_bias * dt

        # Calculate Kalman gain
        self.S = self.P[0][0] + self.R_measure
        self.K[0] = self.P[0][0] / self.S
        self.K[1] = self.P[1][0] / self.S

        # Update estimate with measurement zk (new_angle)
        self.y = new_angle - self.angle
        self.angle += self.K[0] * self.y
        self.bias += self.K[1] * self.y

        # Update covariance matrix
        P00_temp = self.P[0][0]
        P01_temp = self.P[0][1]

        self.P[0][0] -= self.K[0] * P00_temp
        self.P[0][1] -= self.K[0] * P01_temp
        self.P[1][0] -= self.K[1] * P00_temp
        self.P[1][1] -= self.K[1] * P01_temp

        return self.angle, self.bias


In [34]:
ACCEL_SENSITIVITY = 16384
GYRO_SENSITIVITY = 131
def convert_raw_data(ax, ay, az, gx, gy, gz):
    ax = ax / ACCEL_SENSITIVITY
    ay = ay / ACCEL_SENSITIVITY
    az = az / ACCEL_SENSITIVITY

    gx = gx / GYRO_SENSITIVITY
    gy = gy / GYRO_SENSITIVITY
    gz = gz / GYRO_SENSITIVITY

    return ax, ay, az, gx, gy, gz


In [54]:
#Mini Robot Experiment
ser = serial.Serial(port, baud_rate)
sleep(5)


input("Enter any character and press enter")
ser.write(b"rrrrrrrrrr\r\n")
ser.flush()

# Create a CSV file and write headers
csv_file = open('Datasets/TR4.csv', 'w')
csv_writer = csv.writer(csv_file)
# csv_writer.writerow(['Timestamp', 'Increment', 'az1', 'Az1_Angle', 'KalAngleX1', 'az2', 'Az2_Angle', 'KalAngleX2'])
csv_writer.writerow(['Timestamp', 'Increment', 'ax1', 'KalAngleX1', 'ManualX1', 'ay1', 'KalAngleY1', 'az1', 'KalAngleZ1', 'ax2', 'KalAngleX2', 'ManualX2', 'ay2', 'KalAngleY2', 'az2', 'KalAngleZ2'])

try:
    for _ in range(25):
        dt = 0.01
        increment = _ * 7.2
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

        if not any (sensor_data := get_next_sensors_data()):
            csv_writer.writerow([current_time, increment, 0, 0, 0, 0, 0, 0])
            continue
        
        ax1, ay1, az1, gx1, gy1, gz1, ax2, ay2, az2, gx2, gy2, gz2 = sensor_data

        ax1, ay1, az1, gx1, gy1, gz1 = convert_raw_data(ax1, ay1, az1, gx1, gy1, gz1)
        ax2, ay2, az2, gx2, gy2, gz2 = convert_raw_data(ax2, ay2, az2, gx2, gy2, gz2)

        # Initialize Kalman filters for both sensors
        kf1 = KalmanFilter()
        kf2 = KalmanFilter()

        # Calculate angle from acceleration
        az1_angle = (((az1) * 9.8)/0.06) * dt * dt
        ax1_angle = (((ax1) * 9.8)/0.06) * dt * dt
        az2_angle = (((az2) * 9.8)/0.06) * dt * dt
        ax2_angle = (((ax2) * 9.8)/0.06) * dt * dt

        # Estimate angles for both sensors
        angle_x1, bias_x1 = kf1.update(np.arctan2(ay1, az1), gx1, dt)
        angle_x1 = angle_x1 * (180.0 / np.pi)

        angle_y1, bias_y1 = kf1.update(np.arctan2(ax1, az1), gy1, dt)
        angle_y1 = angle_y1 * (180.0 / np.pi)

        angle_z1, bias_z1 = kf1.update(np.arctan2(ax1, ay1), gz1, dt)
        angle_z1 = angle_z1 * (180.0 / np.pi)

        angle_x2, bias_x2 = kf2.update(np.arctan2(ay2, az2), gx2, dt)
        angle_x2 = angle_x2 * (180.0 / np.pi)

        angle_y2, bias_y2 = kf2.update(np.arctan2(ax2, az2), gy2, dt)
        angle_y2 = angle_y2 * (180.0 / np.pi)

        angle_z2, bias_z2 = kf2.update(np.arctan2(ax2, ay2), gz2, dt)
        angle_z2 = angle_z2 * (180.0 / np.pi)

        # Pose of sensor 1
        pose1 = {
            'angle_x': angle_x1,
            'angle_y': angle_y1,
            'angle_z': angle_z1
        }

        # Pose of sensor 2
        pose2 = {
            'angle_x': angle_x2,
            'angle_y': angle_y2,
            'angle_z': angle_z2
        }

        # print("Pose of Sensor 1:", pose1)
        # print("Pose of Sensor 2:", pose2)

        csv_writer.writerow([current_time, increment, ax1, angle_x1, ax1_angle, ay1, angle_y1, az1, angle_z1, ax2, angle_x2, ax2_angle, ay2, angle_y2, az2, angle_z2])
 
except KeyboardInterrupt:
    ser.close()
#except ZeroDivisionError:
    #print("Error: Division by zero is not allowed.")
finally:
    csv_file.close()

In [74]:
#Orthosis
port = 'COM3'  # Replace with the appropriate serial port
baud_rate = 9600
ser = serial.Serial(port, baud_rate)
sleep(5)


input("Enter any character and press enter")
ser.write(b"rrrrrrrrrr\r\n")
ser.flush()

# Create a CSV file and write headers
csv_file = open('Datasets/FINAL4.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Timestamp', 'KalmanXSensor1', 'ManualXSensor1', 'KalmanYSensor1','ManualYSensor1', 'KalmanXSensor2', 'ManualXSensor2', 'KalmanYSensor2', 'ManualYSensor2', 'KalmanXSensor3', 'ManualXSensor3', 'KalmanYSensor3', 'ManualYSensor3', 'PIP-X', 'PIP-Y', 'PIP-Z', 'DIP-X', 'DIP-Y', 'DIP-Z', 'Tip-X', 'Tip-Y', 'Tip-Z'])

try:
    while True:
        dt =  0.01
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        values = [None] * 3

        for _ in range(3):
            # Read data from serial port
            while not (line := ser.read_until()).startswith(b'Sensor'):
                pass

            _, sensor_index, *_, ax, _, ay, _, az, _, gx, _, gy, _, gz = line.rsplit()
            values[int(sensor_index)] = tuple(map(int, (ax, ay, az, gx, gy, gz)))

        # Unpack the data into individual variables
        (ax1, ay1, az1, gx1, gy1, gz1), (ax2, ay2, az2, gx2, gy2, gz2), (ax3, ay3, az3, gx3, gy3, gz3) = values

        ax1, ay1, az1, gx1, gy1, gz1 = convert_raw_data(ax1, ay1, az1, gx1, gy1, gz1)
        ax2, ay2, az2, gx2, gy2, gz2 = convert_raw_data(ax2, ay2, az2, gx2, gy2, gz2)
        ax3, ay3, az3, gx3, gy3, gz3 = convert_raw_data(ax3, ay3, az3, gx3, gy3, gz3)

        # Initialize Kalman filters for both sensors
        kf1 = KalmanFilter()
        kf2 = KalmanFilter()
        kf3 = KalmanFilter()

        # Estimate angles for both sensors
        angle_x1, bias_x1 = kf1.update(np.arctan2(ay1, az1), gx1, dt)
        manual_angle_x1 = np.arctan2(ay1, az1)
        angle_y1, bias_y1 = kf1.update(np.arctan2(ax1, az1), gy1, dt)
        manual_angle_y1 = np.arctan2(ax1, az1)
        angle_z1, bias_z1 = kf1.update(np.arctan2(ax1, ay1), gz1, dt)
        manual_angle_z1 = np.arctan2(ax1, ay1)

        angle_x2, bias_x2 = kf2.update(np.arctan2(ay2, az2), gx2, dt)
        manual_angle_x2 = np.arctan2(ay2, az2)
        angle_y2, bias_y2 = kf2.update(np.arctan2(ax2, az2), gy2, dt)
        manual_angle_y2 = np.arctan2(ax2, az2)
        angle_z2, bias_z2 = kf2.update(np.arctan2(ax2, ay2), gz2, dt)
        manual_angle_z2 = np.arctan2(ax2, ay2)

        angle_x3, bias_x3 = kf3.update(np.arctan2(ay3, az3), gx3, dt)
        manual_angle_x3 = np.arctan2(ay3, az3)
        angle_y3, bias_y3 = kf3.update(np.arctan2(ax3, az3), gy3, dt)
        manual_angle_y3 = np.arctan2(ax3, az3)
        angle_z3, bias_z3 = kf3.update(np.arctan2(ax3, ay3), gz3, dt)
        manual_angle_z3 = np.arctan2(ax3, ay3)


        # Pose of sensor 1
        pose1 = {
            'angle_x': angle_x1,
            'angle_y': angle_y1,
            'angle_z': angle_z1
        }

        # Pose of sensor 2
        pose2 = {
            'angle_x': angle_x2,
            'angle_y': angle_y2,
            'angle_z': angle_z2
        }

        J1x = 0
        J1y = 0
        J1z = 0
            #FINGER 1
        F1J2x = L2 * np.cos(angle_x1) * np.cos(angle_x2)
        F1J2y = L2 * np.sin(angle_x1) * np.cos(angle_x2)
        F1J2z = L2 * np.sin(angle_x2)

        F1J3x = np.cos(angle_x1) * (L2 * np.cos(angle_x2) + L3 * np.cos(angle_x2 + angle_x3))
        F1J3y = np.sin(angle_x1) * (L2 * np.cos(angle_x2) + L3 * np.cos(angle_x2 + angle_x3))
        F1J3z = L2 * np.sin(angle_x2) + L3 * np.sin(angle_x2 + angle_x3)

        F1J4x = (np.cos(angle_x1)) * (L2 * np.cos(angle_x2) + L3 * np.cos(angle_x2 + angle_x3) + L4 * np.cos(angle_x2 + angle_x3))
        F1J4y = (np.sin(angle_x1)) * (L2 * np.sin(angle_x2) + L3 * np.sin(angle_x2 + angle_x3) + L4 * np.sin(angle_x2 + angle_x3))
        F1J4z = L2 * np.sin(angle_x3) + L3 * np.sin(angle_x2 + angle_x3) + L4 * np.sin(angle_x3)
        
        angle_x1 = angle_x1 * (180.0 / np.pi)
        manual_angle_x1 =  manual_angle_x1 * (180.0 / np.pi)

        angle_y1 = angle_y1 * (180.0 / np.pi)
        manual_angle_y1 = manual_angle_y1 * (180.0 / np.pi)

        angle_z1 = angle_z1 * (180.0 / np.pi)

        angle_x2 = angle_x2 * (180.0 / np.pi)
        manual_angle_x2 = manual_angle_x2 * (180.0 / np.pi)

        angle_y2 = angle_y2 * (180.0 / np.pi)
        manual_angle_y2 = manual_angle_y2 * (180.0 / np.pi)

        angle_z2 = angle_z2 * (180.0 / np.pi)

        angle_x3 = angle_x3 * (180.0 / np.pi)
        manual_angle_x3 = manual_angle_x3 * (180.0 / np.pi)

        angle_y3 = angle_y3 * (180.0 / np.pi)
        manual_angle_y3 = manual_angle_y3 * (180.0 / np.pi)
            
        data_timestamp = [current_time, angle_x1, manual_angle_x1, angle_y1, manual_angle_y1, angle_x2, manual_angle_x2, angle_y2, manual_angle_y2, angle_x3, manual_angle_x3, angle_y3, manual_angle_y3, F1J2x, F1J2y, F1J2z, F1J3x, F1J3y, F1J3z, F1J4x, F1J4y, F1J4z]
        csv_writer.writerow(data_timestamp)
        print(data_timestamp)
except KeyboardInterrupt:
    ser.close()
#except ZeroDivisionError:
    #print("Error: Division by zero is not allowed.")
finally:
    csv_file.close()

['2024-08-25 07:43:37.535', 71.52838459371183, 33.690067525979785, 70.72458146848014, 0.0, 0.007931644723731654, -2.4394759457983963, 0.0033295760821354984, -20.02127689850445, -0.008607964012837609, 13.531012640049973, -0.022849710166951486, -21.385823804469588, 0.9505044316012885, 2.845442171795496, 0.0004152999452562071, 2.0594262790177718, 6.165124737134472, 0.00037398595089340445, 2.693095906112905, 0.0003323267387527457, -0.0007925005611229311]
['2024-08-25 07:43:38.850', 71.27916542003341, 33.690067525979785, 69.64298151872995, -81.86989764584403, 71.50591971773345, 45.0, 71.93189333347166, -89.50178838738636, 71.49154057616931, -90.0, 73.03365455467959, 90.0, 0.3054294943809145, 0.9012732646083245, 2.845069300819309, -0.5916884719956289, -1.7459774206401413, 4.951545781546294, -1.1043273099250823, 5.829594872000969, 6.847860463167041]
['2024-08-25 07:43:39.126', 36.53754914643907, 87.40813329476501, 118.25692026220717, 86.53177074108285, 71.32228453712831, 45.0, 71.874995332470

ValueError: not enough values to unpack (expected at least 13, got 5)